# Lab 5: Statistical Distributions in Behavioral Genetics & Substance Use

**Learning Objectives**  
By the end of this lab, you will be able to:  

1. **Standardize continuous variables and assess normality** using z-scores and Q-Q plots (quantile–quantile plots).  
2. **Calculate probabilities from a normal distribution** for standardized behavioral measures.  
3. **Apply the binomial distribution** to model probabilities of binary outcomes (e.g. substance use) in small samples.  
4. **Use the Poisson distribution** to simulate counts of rare events (e.g. opioid overdoses) in a population given a known rate.  
5. **Simulate Bernoulli trials** to understand single-event outcomes (e.g. whether an individual has used a substance) and their probabilities.

## Setup

Run the setup cell below to import libraries and load the dataset for Lab 4/5 (a synthetic ABCD-derived dataset of ~11,000 young adults). This dataset includes cognitive measures, polygenic scores (PGS), and substance use indicators. We’ll use pandas for data handling, numpy for calculations, and matplotlib for plotting.


In [ ]:
from pathlib import Path
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm

# Configure display options
pd.set_option('display.max_columns', None)

FIG_DIR = Path("figures")
FIG_DIR.mkdir(exist_ok=True)

def savefig(name, dpi=150):
    """Save current Matplotlib figure to ./figures/<name>.png"""
    plt.tight_layout()
    plt.savefig(FIG_DIR / name, dpi=dpi)
    plt.close()

# Load the Lab4/Lab5 dataset
df = pd.read_csv('___') # TODO: insert the dataset filename here

# Display shape and first few rows
print(f"Dataset contains {df.shape[0]} rows and {df.shape[1]} columns.")
df.head()


### Activity 1. Normal Distribution I: Standardization and Q-Q Plots

**Demo:** In this first activity, we examine a cognitive measure and test if it follows a **normal distribution**. We’ll use the NIH Toolbox Cognition **Composite Score** (`nc_y_nihtb__comp__tot__agecor_score`), which is an age-corrected cognitive performance score (expected to be roughly normally distributed with mean around 100). We will **standardize** this variable to z-scores and create a **Q-Q plot** to check normality. 

*Standardization* means converting values to units of standard deviations from the mean: `z = (x – mean) / std`. This yields a distribution with mean 0 and SD 1. After standardizing, a **Q-Q plot** (quantile–quantile plot) will compare the sorted data values (quantiles) to theoretical quantiles from a standard normal distribution. If the cognitive scores are normally distributed, the points in the Q-Q plot should lie approximately on a straight diagonal line.

First, let’s compute the mean and standard deviation of the composite scores, convert them to z-scores, and output a few examples:


In [ ]:
# Extract the NIH Toolbox composite score and drop any missing values (including 999 as missing)
scores = df['nc_y_nihtb__comp__tot__agecor_score']
scores = scores[(scores != 999) & (~scores.isna())]

# Calculate mean and standard deviation
mean_score = scores.mean()
std_score = scores.std()

# Standardize to z-scores
z_scores = (scores - mean_score) / std_score

print(f"Mean (raw composite) = {mean_score:.2f}, SD = {std_score:.2f}")
print("First 5 z-scores:", np.round(z_scores.head(5), 2).tolist())

Now we’ll plot a Q-Q plot for the z-scores. We expect the points to roughly follow the $y=x$ line if the data are normal. We use `scipy.stats.probplot` to generate the plot:


In [ ]:
import scipy.stats as stats

# Create a Q-Q plot of the z-scores against a theoretical normal distribution
plt.figure()
stats.probplot(z_scores, dist="norm", plot=plt)
plt.title("Normal Q-Q Plot: NIH Toolbox Composite Score")
plt.xlabel("Theoretical Quantiles")
plt.ylabel("Sample Quantiles")
savefig("qq_plot_nih_toolbox_composite_score.png")
plt.show()


### Interpreting this Normal Q–Q plot

**What a Q–Q plot shows:** If the dots fall on the red line, the data look like a Normal (bell-shaped) distribution. Systematic bends or flat “shelves” mean the data depart from Normal.

**What we see here:**

* **Center looks good.** Most points in the middle track the red line closely → the bulk of scores are roughly bell-shaped.
* **Right side “shelf.”** The dots flatten out around ~2 on the vertical axis (a little horizontal plateau). That’s a **ceiling effect**—many people are hitting or bunching near a top score, so the upper tail is **lighter/shorter** than a true Normal.
* **Left tail dip.** A few low values sit below the line (around −3 to −3.5), suggesting a small departure at the very low end (possible floor clumping or mild skew).

**What this means for analysis:**

* For typical summaries and many tests, it’s fine to report **mean and SD** (the center behaves well).
* Because the tails aren’t perfectly Normal—especially the **ceiling**—also report a **median and IQR**, and be cautious drawing conclusions about **extreme** scores.
* If your question depends on the high end (e.g., distinguishing top performers), note the ceiling effect and consider a robustness check (e.g., trimmed mean) or simply state the limitation.

### Your Turn: NIH Toolbox vs. Cognitive PGS Normality

Now it’s your turn to repeat this process with a different variable: the **Cognitive Polygenic Score** (`pgs_cog_std`). This variable is a standardized polygenic score (PGS) for cognitive ability (higher values might indicate higher genetic propensity for cognitive performance). 

Follow these steps:  
1. Calculate the mean and standard deviation of `pgs_cog_std` (note: since it’s labeled `_std`, it may already have mean ~0 and SD ~1, but you can verify).  
2. Convert the PGS values to z-scores (or if already standardized, you can use them as-is).  
3. Create a Q-Q plot for `pgs_cog_std` similar to the demo above.  
4. Examine the Q-Q plot to determine if the cognitive PGS appears normally distributed (do the points fall on a straight line, or do you see systematic curves/deviations?).


In [ ]:
# YT1.1 QQ Plot for Cognitive PGS

# Step 1. Extract and clean (TODO: students fill the column name)
pgs = df['___']   # TODO: replace ___ with 'pgs_cog_std'
pgs = pgs[(pgs != 999) & (~pgs.isna())]

# Step 2. Standardize (fill in formula)
mean_pgs = pgs.mean()
std_pgs = pgs.std(ddof=1)
z_pgs = (pgs - ___) / ___   # TODO: subtract mean, divide by sd

print(f"Mean (pgs_cog_std) = {mean_pgs:.2f}, SD = {std_pgs:.2f}")

# Step 3. QQ plot (students write the actual plot commands)
plt.figure()
# TODO: call scipy.stats.probplot with z_pgs, dist="norm", and plot=plt
# TODO: set a title and axis labels
# Example titles/labels: 
#   Title: "QQ Plot — Cognitive PGS (z)"
#   X-axis: "Theoretical Quantiles"
#   Y-axis: "Sample Quantiles"

# TODO: save figure with filename "Q1a_QQ_pgs_z.png"
plt.show()

# Step 4. Print a quick check of first 5 z-scores (to confirm they look standardized)
print("First 5 z-scores:", np.round(z_pgs.head(5), 2).tolist())


This code calculates the proportion of individuals with z-scores above 1.65 (top ~5%) for both the NIH Toolbox composite and the cognitive PGS.

In [ ]:
# Proportion above z = 1.65 (~top 5%) for both variables
z_cut = 1.65
prop_above_demo = (z_scores >= z_cut).mean()
prop_above_yourturn = (z_pgs >= z_cut).mean()
print("Proportion above z=1.65 (NIH Toolbox):", round(prop_above_demo, 3))
print("Proportion above z=1.65 (PGS):", round(prop_above_yourturn, 3))

### Exploring the Relationship: Bivariate EDA of Cognitive PGS and NIH Toolbox Composite

Now that we've looked at each variable separately, let's explore how they relate to each other. This step is called **bivariate exploratory data analysis (EDA)**. We'll examine whether individuals with higher cognitive polygenic scores (PGS) also tend to have higher cognitive performance (as measured by the NIH Toolbox Composite Score).

Here's what we'll do:
- **Clean and standardize** both variables to ensure they're on the same scale and missing values are handled.
- **Calculate correlations** (Pearson and Spearman) to quantify the strength and direction of the association.
- **Visualize the relationship** with a scatterplot, showing how cognitive PGS and performance move together across individuals.
- **Summarize trends** by plotting average performance within each decile (10% group) of PGS, giving a clearer sense of how performance changes across the PGS spectrum.

This analysis helps us see not just if there's a relationship, but also how strong it is and what its pattern looks like across the range of genetic scores.

In [ ]:
# --- Bivariate EDA: Cognitive PGS vs NIH Toolbox Composite ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# 1) Clean and standardize both variables
x = df['pgs_cog_std']
y = df['nc_y_nihtb__comp__tot__agecor_score']

# Treat 999 as missing if present, then drop NaNs
mask = (~x.isna()) & (x != 999) & (~y.isna()) & (y != 999)
x = x[mask].astype(float)
y = y[mask].astype(float)

# Standardize outcome (PGS is already ~std, but standardize both for symmetry)
zx = (x - x.mean()) / x.std()
zy = (y - y.mean()) / y.std()

# 2) Correlations
pearson_r, pearson_p = stats.pearsonr(zx, zy)
spearman_rho, spearman_p = stats.spearmanr(zx, zy)

print(f"Pearson r = {pearson_r:.3f} (p = {pearson_p:.3g})")
print(f"Spearman ρ = {spearman_rho:.3f} (p = {spearman_p:.3g})")
print("Interpretation tip: with both variables z-scored, Pearson r ≈ SD change in NIH score per 1 SD of PGS.")

# 3) Scatter with light alpha
plt.figure()
plt.scatter(zx, zy, s=10, alpha=0.25)
plt.xlabel("Cognitive PGS (z)")
plt.ylabel("NIH Toolbox Composite (z)")
plt.title("PGS vs Cognitive Performance (z–z scatter)")
plt.axhline(0, lw=0.5)
plt.axvline(0, lw=0.5)
plt.show()

# 4) Quantile-bin means (deciles of PGS) as a simple trend summary
q = pd.qcut(zx, 10, labels=False, duplicates='drop')
decile_means = pd.DataFrame({"zx": zx, "zy": zy, "q": q}).groupby("q").agg(
    mean_pgs=("zx","mean"),
    mean_perf=("zy","mean"),
    se_perf=("zy", lambda s: s.std(ddof=1)/np.sqrt(len(s)))
).reset_index()

plt.figure()
plt.errorbar(decile_means["mean_pgs"], decile_means["mean_perf"],
             yerr=decile_means["se_perf"], fmt='o-', capsize=3)
plt.xlabel("Cognitive PGS (decile means, z)")
plt.ylabel("NIH Toolbox Composite (decile means, z)")
plt.title("Decile plot: mean performance by PGS decile (±SE)")
plt.axhline(0, lw=0.5)
plt.axvline(0, lw=0.5)
savefig("decile_plot_pgs_vs_nih_toolbox.png")
plt.show()

# 5) Quick takeaway string
slope_like = pearson_r  # with z–z, the regression slope would equal r
print(f"Quick takeaway: A 1 SD increase in PGS is associated with about {slope_like:.2f} SD higher performance (by Pearson r).")


### From visualization to summary numbers

The decile plot gave us an intuitive picture: as cognitive PGS increases, average NIH Toolbox scores also increase in a fairly smooth, monotonic way.

To communicate this relationship clearly, it helps to pair two complementary summaries:

- **Pearson r** — a compact statistic showing how strongly two standardized variables move together.
- **Δ₁₀–₁ (decile gap)** — the average difference in outcome between the top and bottom deciles of PGS, which provides an intuitive effect size.

In the next code cell, we’ll calculate both side by side.

In [ ]:
# Assumes df is loaded; 999 marks non-response; both vars exist.
# This cell recomputes everything in case prior cells weren’t run.

import numpy as np
import pandas as pd
from scipy import stats

x = df['pgs_cog_std']
y = df['nc_y_nihtb__comp__tot__agecor_score']

mask = (~x.isna()) & (x != 999) & (~y.isna()) & (y != 999)
x = x[mask].astype(float)
y = y[mask].astype(float)

# z–z scaling for interpretability
zx = (x - x.mean()) / x.std(ddof=1)
zy = (y - y.mean()) / y.std(ddof=1)

# 1) Correlations
pearson_r, pearson_p = stats.pearsonr(zx, zy)
spearman_rho, spearman_p = stats.spearmanr(zx, zy)

# 2) Decile gap Δ10–1 (mean outcome in top vs bottom PGS decile)
q = pd.qcut(zx, 10, labels=False, duplicates='drop')  # 0..9
dm = pd.DataFrame({'zx': zx, 'zy': zy, 'q': q}).groupby('q')['zy'].mean()
delta_10_1 = dm.loc[dm.index.max()] - dm.loc[dm.index.min()]

print(f"Pearson r = {pearson_r:.3f} (p={pearson_p:.3g})")
print(f"Spearman ρ = {spearman_rho:.3f} (p={spearman_p:.3g})")
print(f"Δ10–1 (top–bottom decile gap, in SD) = {delta_10_1:.2f}")

print(
    "One-liner: PGS and performance are positively related "
    f"(r={pearson_r:.2f}); top-decile vs bottom-decile differs by "
    f"≈ {delta_10_1:.2f} SD."
)


### Communicating the association: report **r** + Δ₁₀–₁ (decile gap)

**Why both?**

- **Pearson r** (with both variables z-scored) is a compact, unitless summary of linear association. Interpretable as: “SD change in performance per 1 SD of PGS.”
- **Δ₁₀–₁** is the top–bottom decile gap in outcome (in SD units if you z-scored the outcome). It’s an intuitive “distance” for non-technical audiences.

**How to report (1–2 sentences):**

> “PGS and performance are positively related (Pearson r = ___). On average, students in the top PGS decile score ___ SD higher than those in the bottom decile (Δ₁₀–₁ = ___ SD).”

### Activity 2. Normal Distribution II: Normal Probabilities

**Demo:** Next, we will use the normal distribution to answer probability questions about a behavioral measure. Specifically, let’s consider a **delay discounting** measure, `nc_y_ddis__1mo__indifpt_prop`, which represents a 1-month delay discounting indifference point (a proportion indicating how much of a reward a person would take now vs. in 1 month). This is a continuous measure of impulsivity. We’ll assume (after standardization) that it roughly follows a normal distribution. Using the **standard normal CDF**, we can find probabilities of certain outcomes (e.g. what fraction of individuals fall above or below a certain threshold).

First, standardize the delay discounting score to get z-scores (mean 0, SD 1):


In [ ]:
# Delay discounting 1-month proportion score
dd_score = df['nc_y_ddis__1mo__indifpt_prop'].dropna()
mean_dd = dd_score.mean()
std_dd = dd_score.std()
dd_z = (dd_score - mean_dd) / std_dd

print(f"Mean delay discounting score = {mean_dd:.3f}, SD = {std_dd:.3f}")


Now, using the standard normal distribution, we can calculate some probabilities. For example, **what proportion of people have a delay discounting z-score greater than 1?** (i.e., more than one standard deviation above the mean, indicating unusually high impulsivity). Similarly, we can find the probability of being **lower than –1** (very low impulsivity) or beyond **+2** (extremely high impulsivity). We’ll use `scipy.stats.norm.cdf` for cumulative probabilities:


In [ ]:
# Probability of being more than 1 SD above the mean (Z > 1)
prob_above1 = 1 - norm.cdf(1)   # = P(Z > 1) = 1 - P(Z <= 1)
# Probability of being more than 1 SD below the mean (Z < -1)
prob_below_neg1 = norm.cdf(-1)  # = P(Z < -1)
# Probability of being more than 2 SD above the mean (Z > 2)
prob_above2 = 1 - norm.cdf(2)

print(f"P(Z > 1)  = {prob_above1:.3f} (proportion above +1 SD)")
print(f"P(Z < -1) = {prob_below_neg1:.3f} (proportion below -1 SD)")
print(f"P(Z > 2)  = {prob_above2:.3f} (proportion above +2 SD)")


The output gives us approximate probabilities assuming a normal distribution. Recall the 68–95–99.7 rule for a normal distribution: ~16% of data are above +1 SD, ~16% below –1 SD, and only about 2.3% are above +2 SD (or below –2 SD).

From our calculations: about **0.159 (~15.9%)** of individuals have $Z>1$ (higher-than-average impulsivity), the same ~15.9% have $Z<-1$ (very low impulsivity), and only **~0.023 (2.3%)** have $Z>2$ (extremely high impulsivity). These align with our expectations for a normal distribution.

---

### Visualizing the Normal Distribution and Our Data

To make these concepts more concrete, let's use several visualizations to connect the probability calculations to both the theoretical normal distribution and our actual data:

1. **Standard Normal PDF with Shaded Area:**  
   This plot shows the bell curve of the standard normal distribution, with the right tail shaded above a chosen z-score (e.g., the 90th percentile). This visually represents the probability of being above that threshold.

2. **Standard Normal CDF with Percentile Marker:**  
   Here, we plot the cumulative distribution function (CDF), which shows the probability of being less than or equal to a given z-score. The chosen percentile and its corresponding z-value are marked, helping you see how percentiles relate to z-scores.

3. **Empirical Histogram of z-scores with Normal PDF Overlay:**  
   This histogram displays the distribution of z-scores from our actual delay discounting data, overlaid with the standard normal curve. This helps you compare the real data’s shape to the theoretical normal distribution.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm

# --- 0) Prepare Delay Discounting data and z-scores (standalone) ---
y = df['nc_y_ddis__1mo__indifpt_prop']
y = y[(y != 999) & (~y.isna())].astype(float)

mu_y, sd_y = y.mean(), y.std(ddof=1)
zy = (y - mu_y) / sd_y

# Choose a percentile and its z-cut (example: 90th)
pct = 0.90
z0  = norm.ppf(pct)       # ~1.2816
x0  = mu_y + z0*sd_y      # raw-score cut in the variable's units

# --- 1) Standard Normal PDF with shaded right-tail area ---
zgrid = np.linspace(-4, 4, 800)
pdf   = norm.pdf(zgrid)
area  = 1 - norm.cdf(z0)

plt.figure()
plt.plot(zgrid, pdf)
plt.fill_between(zgrid, 0, pdf, where=(zgrid>=z0), alpha=0.3)
plt.axvline(z0, ls='--', lw=1)
plt.title(f"Standard Normal PDF — Shaded P(Z ≥ {z0:.2f}) ≈ {area:.03f}")
plt.xlabel("z")
plt.ylabel("density")
plt.show()

# --- 2) Standard Normal CDF with percentile marker ---
cdf = norm.cdf(zgrid)

plt.figure()
plt.plot(zgrid, cdf)
plt.axvline(z0, ls='--', lw=1)
plt.axhline(pct, ls='--', lw=1)
plt.scatter([z0], [pct])
plt.title(f"Standard Normal CDF — {int(pct*100)}th pct at z≈{z0:.2f}")
plt.xlabel("z")
plt.ylabel("P(Z ≤ z)")
plt.show()

# --- 3) Empirical histogram of z-scores with Normal PDF overlay ---
plt.figure()
plt.hist(zy, bins=40, density=True, alpha=0.35)
plt.plot(zgrid, pdf, lw=2)
plt.axvline(z0, ls='--', lw=1, label=f"{int(pct*100)}th pct z≈{z0:.2f}")
plt.title("Delay Discounting z-scores: Histogram + Normal PDF")
plt.xlabel("z-score of nc_y_ddis__1mo__indifpt_prop")
plt.ylabel("density")
plt.legend()
plt.show()

print(f"{int(pct*100)}th percentile cut:")
print(f"  z ≈ {z0:.3f}")
print(f"  raw score ≈ {x0:.4f} (x = μ + z·σ)")
print(f"  Right-tail area P(Z ≥ {z0:.2f}) ≈ {area:.3f}")


### YT2 — Student Choice: Normal Tail Probabilities

Pick **one** continuous variable from the lists below, then:  
1. Recode ABCD missing codes (`777, 888, 999`) to `NaN`.  
2. Compute mean/SD and (re)standardize to z-scores.  
3. Report Normal-model tail probabilities at three cutoffs (mirrors the demo).

**Normal-friendly (good Normal fit):**  
- `pgs_externalizing_std` (Externalizing PGS, z)  
- `pgs_cog_std` (Cognitive PGS, z)  
- `nc_y_nihtb__comp__tot__agecor_score` (NIH Toolbox composite) → z-score yourself  
- `mh_y_upps__nurg_sum` (UPPS Negative Urgency) → z-score yourself  

**Stress-test (skew/ceiling/zeros):**  
- `nc_y_ddis__1mo__indifpt_prop` (Delay Discounting %) → z-score yourself  
- `nc_y_flnkr__incongr_acc` (Flanker accuracy; ceiling near 1.0) → z-score yourself  
- `mh_p_cbcl__synd__ext_sum` (CBCL Externalizing; zero-inflated) → z-score yourself  

**What to print (exact labels):**  
- `VAR_CHOSEN = ...`  
- `N_VALID = ...`  
- `MEAN = ... ; SD = ...`  
- `P(Z > 1) = ...   (proportion above +1 SD)`  
- `P(Z < -1) = ...  (proportion below -1 SD)`  
- `P(Z > 2) = ...   (proportion above +2 SD)`  

*(Optional: also print empirical tail fractions from your z-scores to compare model vs. data.)*  

**🤖 Copilot prompts (optional):**  
- “Show Python code to z-score a pandas Series and print mean/SD and N after dropping 777/888/999.”  
- “Compute Normal tail probabilities P(Z>1), P(Z<−1), P(Z>2) using `scipy.stats.norm`.”  
- “Compare empirical tail fractions from my z-scores to Normal-model expectations and print both.”  


In [ ]:
# YT2 — Student choice: Normal tail probabilities (mirrors the demo; fill the blanks)

import numpy as np
import pandas as pd
from scipy.stats import norm

# 1) CHOOSE ONE VARIABLE from the lists above
VAR_NAME = '___'  # e.g., 'pgs_cog_std' or 'nc_y_nihtb__comp__tot__agecor_score'

# 2) Extract and clean (ABCD-style missing → NaN)
series = df[___]   # TODO: insert VAR_NAME here
series = series.replace({777: np.nan, ___: np.nan, ___: np.nan}).astype(float).dropna()

# 3) Compute mean/SD and (re)standardize to z
mean_val = series.mean()
sd_val   = series.std(ddof=___)     # TODO: set ddof
z_vals   = (series - ___) / ___     # TODO: subtract mean_val, divide by sd_val

print(f"VAR_CHOSEN = {VAR_NAME}")
print(f"N_VALID = {z_vals.shape[0]}")
print(f"MEAN = {mean_val:.3f} ; SD = {sd_val:.3f}")

# 4) Normal-model tail probabilities (same as demo)
prob_above1     = 1 - norm.cdf(___)   # TODO: P(Z > 1)
prob_below_neg1 = norm.cdf(-1)        # P(Z < -1)
prob_above2     = 1 - norm.cdf(2)     # P(Z > 2)

print(f"P(Z > 1) = {prob_above1:.3f} (proportion above +1 SD)")
print(f"P(Z < -1) = {prob_below_neg1:.3f} (proportion below -1 SD)")
print(f"P(Z > 2) = {prob_above2:.3f} (proportion above +2 SD)")

# (Optional) Empirical tails from your standardized sample
emp_above1  = (z_vals >= 1).mean()
emp_below_1 = (z_vals <= -1).mean()
emp_above2  = (z_vals >= 2).mean()
print(f"[Empirical] P(Z > 1) = {emp_above1:.3f} ; P(Z < -1) = {emp_below_1:.3f} ; P(Z > 2) = {emp_above2:.3f}")


## YT3 — Visualizing Normal Theory vs. Your Data

You’ll recreate three visuals from the demo for your YT2 variable:

1. **Standard Normal PDF with shaded right tail at your chosen percentile’s z-cut.**
2. **Standard Normal CDF with percentile marker (vertical and horizontal reference lines).**
3. **Empirical histogram of your z-scores overlaid with the Standard Normal PDF (compare data vs. theory).**

**Instructions:**
- Use your YT2 objects (`series`, `mean_val`, `sd_val`, `z_vals`).
- Pick a percentile `pct` (e.g., 0.90).
- Compute `z0 = norm.ppf(pct)` (the z-score for your chosen percentile).
- Back-transform to the raw cut: `x0 = mean_val + z0 * sd_val`.
- Reproduce the three plots as shown in the demo.

---

### Copilot prompts (optional):

- “How do I use scipy.stats.norm.ppf to convert a percentile to a z-score and back-transform to a raw score?”
- “Show code to shade the right tail of the Standard Normal PDF beyond a given z0 using fill_between.”

In [ ]:
# YT3 — Visuals (mirror the demo; fill the blanks)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm

# Reuse your YT2 results
zy = pd.Series(z_vals).dropna()   # standardized data (z-scores) from YT2

# --- A) Choose a percentile and compute z-cut + raw cut (fill the blanks) ---
pct = ___                 # e.g., 0.90  (the 90th percentile)
z0  = norm.ppf(___)       # TODO: pass your percentile
x0  = ___ + ___*___       # TODO: mean_val + z0*sd_val  (back-transform to raw)

# --- B) Precompute grids and theory curves ---
zgrid = np.linspace(-4, 4, ___)   # TODO: number of points (e.g., 800)
pdf   = norm.pdf(zgrid)
cdf   = norm.cdf(zgrid)
area  = ___ - norm.cdf(z0)        # TODO: right-tail area (use 1 - norm.cdf(z0))

# --- 1) Standard Normal PDF with shaded right-tail area ---
plt.figure()
plt.plot(zgrid, pdf)
plt.fill_between(zgrid, 0, pdf, where=(zgrid >= z0), alpha=0.3)
plt.axvline(z0, ls='--', lw=1)
# TODO: add a descriptive title showing z0 and area (see demo)
# plt.title(f"Standard Normal PDF — Shaded P(Z ≥ {z0:.2f}) ≈ {area:.03f}")
plt.xlabel("z")
plt.ylabel("density")
plt.show()

# --- 2) Standard Normal CDF with percentile marker ---
plt.figure()
plt.plot(zgrid, cdf)
plt.axvline(z0, ls='--', lw=1)
plt.axhline(pct, ls='--', lw=1)
# TODO: place the marker point at (z0, pct)
# plt.scatter([z0], [pct])
# TODO: add a title showing the percentile and z0 (see demo)
# plt.title(f"Standard Normal CDF — {int(pct*100)}th pct at z≈{z0:.2f}")
plt.xlabel("z")
plt.ylabel("P(Z ≤ z)")
plt.show()

# --- 3) Empirical histogram of z-scores with Normal PDF overlay ---
plt.figure()
plt.hist(zy, bins=___, density=___, alpha=0.35)   # TODO: choose bins (e.g., 40) and set density=True
plt.plot(zgrid, pdf, lw=2)
plt.axvline(z0, ls='--', lw=1, label=f"{int(pct*100)}th pct z≈{z0:.2f}")
plt.title("Your variable (z): Histogram + Normal PDF")
plt.xlabel(f"z-score of {VAR_NAME}")
plt.ylabel("density")
plt.legend()
plt.show()

# Printed summary (like the demo)
print(f"{int(pct*100)}th percentile cut:")
print(f"  z ≈ {z0:.3f}")
print(f"  raw score ≈ {x0:.4f} (x = μ + z·σ)")
print(f"  Right-tail area P(Z ≥ {z0:.2f}) ≈ {area:.3f}")


### Activity 3. Binomial Distribution: Substance Use in a Sample

**Demo:** The **binomial distribution** models the number of “successes” in a fixed number of independent trials, given a constant probability of success on each trial. In our context, we can use a binomial model to answer questions like: *“What is the probability that in a group of $n$ people, exactly $k$ of them have engaged in a certain substance use behavior?”* or *“What is the probability at least one person in the group has tried substance X?”*

Let’s demonstrate this using **alcohol use (sip)**. The variable `su_y_sui__use__alc__sip_001__l` indicates whether the participant has ever had even a sip of alcohol (0 = No, 1 = Yes). First, we need the probability $p$ that any one individual has had an alcohol sip. We can estimate $p$ from our data as the sample proportion of 1s in that column. Then we’ll consider a **small sample** of $n=5$ people (e.g., a group of 5 friends) and use the binomial distribution to calculate probabilities.


**A. Estimate \( p \) from data**

In [ ]:
# Choose the binary variable (0/1; sip of alcohol)
col = "su_y_sui__use__alc__sip_001__l"
x = df[col].dropna().astype(float)
p = x.mean()        # estimated individual probability
n = 5               # sample size (group of 5)

print(f"VARIABLE = {col}")
print(f"p = {p:.3f} ; n = {n}")


#### B. Show the binomial PMF

In [ ]:
k = np.arange(n+1)
pmf = binom.pmf(k, n, p)

plt.figure()
plt.bar(k, pmf, edgecolor="black")
plt.xlabel("k = number in group with a sip (out of n)")
plt.ylabel("Probability")
plt.title(f"Binomial PMF — Alcohol Sip (p={p:.2f}, n={n})")
plt.xticks(k)
plt.show()

# one-line methods caption students can reuse
print(f"Methods: p estimated as sample mean of {col}; Binomial PMF shown for k=0..{n}.")


#### C. Compute key probabilities

In [ ]:
P0 = binom.pmf(0, n, p)         # none
P_ge1 = 1 - P0                   # at least one
k_exact = 3                      # editable: exactly 3
Pk = binom.pmf(k_exact, n, p)

print(f"P(0 of {n})   = {P0:.3f}")
print(f"P(≥1 of {n})  = {P_ge1:.3f}")
print(f"P(exactly {k_exact} of {n}) = {Pk:.3f}")


### Interpretation

Think of the bars as answers to:  
“Out of a group of $n$ people, how many will have had a sip?”  
Each bar at $k$ shows the chance of seeing exactly $k$ people with a sip.

With $p \approx {p:.2f}$ and $n = {n}$, the tallest bar is near $k \approx n p$ (here, around $k = \mathrm{int}(\mathrm{round}(\mathrm{float}(p) \times n))$).  
That tallest bar marks the most likely headcount in a group of size $n$.

The leftmost bar ($k = 0$) is the chance that no one in the group has had a sip:  
$P(X = 0) = {P0:.3f}$.  
Everything to the right of that bar represents “at least one,” which is why  
$1 - P(X = 0) = {P_ge1:.3f}$.

If you look at the bar at $k = {k_exact}$, that single bar corresponds to the chance of seeing exactly that many people with a sip:  
$P(X = {k_exact}) = {Pk:.3f}$.

As $p$ gets larger (each person is more likely), the whole set of bars shifts to the right and $P(X \ge 1)$ gets closer to 1.  
When $p$ is smaller, most of the mass stays on the left, especially the $k = 0$ bar.

### Your Turn: Binomial Model on a Binary Outcome (Student Choice)

**Goal (same as the demo):** Estimate an individual probability \(p\) from a 0/1 column and use the binomial model \(X\sim \text{Binomial}(n,p)\) to compute and **visualize** probabilities for a small group.

**Choose ONE binary variable (0/1):**
- **Core (recommended; no/low missing):**  
  `su_y_sui__use__mj__puff_001__l` (Marijuana puff),  
  `su_y_sui__use__alc__sip_001__l` (Alcohol sip),  
  `su_y_sui__use__nic__puff_001__l` (Nicotine puff/vape).
- **Optional/Advanced (recode 777/999→NaN before computing):**  
  `mh_p_famhx__alc_001__v02`, `mh_p_famhx__drg_001__v02`, `mh_p_famhx__troub_001__v02`.

**What to do (mirror the demo):**
1) Compute \(p\) as the sample mean of your chosen column (after cleaning, if needed).  
2) Set a small group size \(n\) (use \(n=5\) to match the demo).  
3) Make the **Binomial PMF** bar chart for \(k=0..n\) with a clear title/axes.  
4) Compute and print: \(P(X=0)\) (none), \(P(X\ge 1)=1-P(X=0)\) (at least one), and \(P(X=k_{\text{exact}})\) for a chosen \(k\).  
5) Interpret using the figure (where is the tallest bar ≈ \(np\)? Is “none” plausible? How big is “at least one”?).

**🤖 Copilot prompts (optional):**
- “Select a binary column in pandas and recode 777/999 to NaN before computing the mean.”  
- “Use `scipy.stats.binom` to compute a PMF for k=0..n and plot it with matplotlib bars.”  
- “Compute P(X=0), P(X≥1) as a complement, and P(X=k_exact) for a binomial distribution.”


In [ ]:
# YT — Binomial PMF (mirror the demo; fill the blanks)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import binom

# 1) Choose ONE binary variable from the list in the markdown
col = '___'   # e.g., 'su_y_sui__use__mj__puff_001__l' or 'su_y_sui__use__alc__sip_001__l'

# 2) Extract and clean (advanced vars may use 777/999 codes; harmless for core vars)
x = df[col].replace({777: np.nan, 999: np.nan}).dropna().astype(float)

# 3) Estimate p and set group size n
p = x.mean()          # estimated individual probability
n = ___               # TODO: set small sample size (use 5 to mirror the demo)

print(f"VARIABLE = {col}")
print(f"p = {p:.3f} ; n = {n}")

# 4) Binomial PMF and bar chart
k   = np.arange(n + 1)
pmf = binom.pmf(k, n, p)

plt.figure()
plt.bar(k, pmf, edgecolor="black")
plt.xlabel("k = number in group with the behavior (out of n)")
plt.ylabel("Probability")
plt.title(f"Binomial PMF — ___ (p={p:.2f}, n={n})")   # TODO: replace ___ with a short label (e.g., 'Alcohol Sip' or 'Marijuana Puff')
plt.xticks(k)
plt.show()

# One-line methods caption (students can reuse in reflection)
print(f"Methods: p estimated as sample mean of {col}; Binomial PMF shown for k=0..{n}.")

# 5) Key probabilities
P0       = binom.pmf(0, n, p)      # none
P_ge1    = ___                     # TODO: complement of P0 (at least one)
k_exact  = ___                     # TODO: choose an exact count (e.g., 2 or 3)
Pk       = binom.pmf(k_exact, n, p)

print(f"P(0 of {n})   = {P0:.3f}")
print(f"P(≥1 of {n})  = {P_ge1:.3f}")
print(f"P(exactly {k_exact} of {n}) = {Pk:.3f}")


### Activity 4. Poisson Distribution: Simulating Overdose Counts

**Demo:** The **Poisson distribution** is useful for modeling counts of rare events over a given time or space, especially when the events occur independently. In behavioral genetics and public health, we might use a Poisson model for something like the number of **opioid overdose events** in a population within a year. The Poisson distribution has a single parameter $\lambda$ (lambda), which is both the expected value *and* the variance of the distribution. For example, if on average 10 overdoses occur per year in a city, we could model the yearly count of overdoses as $	ext{Pois}(\lambda=10)$.

Let’s simulate a scenario: suppose the **annual overdose rate** is approximately “$\mathbf{0.001}$ per person per year” (this would correspond to about 0.1% risk per person per year, or 1 overdose per 1000 people per year – a rate in the range of reported values for opioid overdoses in some populations). If our synthetic sample has about 10,000 individuals, the expected number of overdoses in one year for this group is $\lambda = 10{,}000 	imes 0.001 = 10$. We will simulate the distribution of the number of overdose events in one year across many hypothetical “repetitions” of that year.


In [ ]:
import numpy as np

# Define rate and population
rate_per_person = 0.001   # 0.1% chance of an overdose per person-year
population = 10000       # consider 10,000 people
lam = rate_per_person * population  # expected number of overdoses in the group per year

# Simulate the number of overdose events over 10,000 hypothetical one-year trials
num_simulations = 10000
counts = np.random.poisson(lam, size=num_simulations)

# Approximate probabilities from simulation for a range of counts
# e.g., frequency of seeing 0 overdoses, 1 overdose, 2 overdoses, ... etc.
vals, freqs = np.unique(counts, return_counts=True)
prob_est = freqs / num_simulations

# Print some example probabilities from the simulation
for k in range(0, 6):  # 0 through 5 overdoses
    p_k = prob_est[vals.tolist().index(k)] if k in vals else 0
    print(f"P(X = {k} overdoses) ≈ {p_k:.3f}")


The above simulation allows us to estimate probabilities like $P(X=0)$, $P(X=1)$, etc., where $X$ = number of overdoses in a year in this population. We expect $P(X=10)$ to be the highest (since $\lambda=10$), and probabilities will drop off for much larger or much smaller counts.

Let’s also visualize the **Poisson distribution** of $X$ with $\lambda = 10$: we’ll plot the probability mass for counts. 


In [ ]:
import matplotlib.pyplot as plt

# Limit to a reasonable range for plotting (say 0 to 20 overdoses)
max_k = 20
vals_plot = vals[vals <= max_k]
probs_plot = prob_est[vals <= max_k]

plt.figure(figsize=(6,4))
plt.bar(vals_plot, probs_plot, width=0.8, color='skyblue', edgecolor='black')
plt.xticks(range(0, max_k+1, 2))
plt.xlabel("Number of opioid overdose events in one year (N=10,000 people)")
plt.ylabel("Probability")
plt.title(f"Distribution of annual overdose counts (λ = {lam:.1f})")
savefig("poisson_overdose_distribution.png")
plt.show()


*Simulated Poisson distribution of yearly opioid overdose counts for a population of 10,000 (assuming an average rate $\lambda=10$ per year).* The bar heights represent the probability of seeing that many overdose events in a year. The distribution is centered around 10 (the expected value) and has a roughly bell-shaped spread: for example, 8, 9, 10, 11 overdoses are the most likely counts in a year, each with around 10–12% probability. Lower counts like 0, 1, 2 are extremely unlikely (less than 0.1% for 0 events, in this simulation), and very high counts (say 20 or more) are also unlikely. This reflects the Poisson property that the variance equals the mean – there’s considerable year-to-year fluctuation, but it’s relatively rare to deviate by more than a few events from the mean in either direction.

### Your Turn: Adjusting Rates or Population in Poisson

Now, experiment with the Poisson model by changing parameters:  

- **Try a different rate:** For instance, what if the overdose rate were higher, say 0.002 per person (0.2%)? Recalculate $\lambda$ for the same population of 10,000 (it would double to 20) and simulate again. How does the distribution change (is it wider, more skewed)? What’s the probability of 0 overdoses now (it will be even smaller)?  

- **Try a different population size:** If we only had 1,000 people instead of 10,000 (keeping the original 0.1% rate), $\lambda$ would drop to 1.0. That yields a $	ext{Pois}(1)$ distribution, which is much more likely to have 0 events in a year. You can simulate this scenario and observe that $P(X=0)$ might be around 37% when $\lambda=1$.  

Feel free to adjust `rate_per_person` or `population` in the code and re-run it. Then inspect the probabilities or plot: see how a larger $\lambda$ (due to higher rate or more people) makes the Poisson distribution spread out more (and the chance of zero events drops), whereas a smaller $\lambda$ makes the distribution concentrate more at the lower counts (including a higher chance of zero events).

**🤖 Copilot prompts (optional):**

 - “Select a binary column in pandas and recode 777/999 to NaN before computing the mean.”
 - “Use scipy.stats.binom to compute a PMF for k=0..n and plot it with matplotlib bars.”
 - “Compute P(X=0), P(X≥1) as a complement, and P(X=k_exact) for a binomial distribution.”

In [ ]:
# Define rate and population
rate_per_person = ____   # e.g., 0.001 for 0.1% chance per person-year
population = ____        # e.g., 10000 people
lam = rate_per_person * population  # expected number of overdoses in the group per year

# Simulate the number of overdose events over ____ hypothetical one-year trials
num_simulations = ____   # e.g., 10000
counts = np.random.poisson(lam, size=num_simulations)

# Approximate probabilities from simulation for a range of counts
vals, freqs = np.unique(counts, return_counts=True)
prob_est = freqs / num_simulations

# Print some example probabilities from the simulation
for k in range(0, ____):  # e.g., 0 through 5 overdoses
    p_k = prob_est[vals.tolist().index(k)] if k in vals else 0
    print(f"P(X = {k} overdoses) ≈ {p_k:.3f}")

import matplotlib.pyplot as plt

# Limit to a reasonable range for plotting (e.g., 0 to 20 overdoses)
max_k = ____
vals_plot = vals[vals <= max_k]
probs_plot = prob_est[vals <= max_k]

plt.figure(figsize=(6,4))
plt.bar(vals_plot, probs_plot, width=0.8, color='skyblue', edgecolor='black')
plt.xticks(range(0, max_k+1, 2))
plt.xlabel(f"Number of opioid overdose events in one year (N={population} people)")
plt.ylabel("Probability")
plt.title(f"Distribution of annual overdose counts (λ = {lam:.1f})")
savefig("poisson_overdose_distribution_adjusted.png")
plt.show()

### Optional Activity 3 Extension: Binomial (with integrated Bernoulli micro-lesson)

**Learning goal.** Model counts of “successes” (yes/no outcomes) across a small group using the Binomial distribution, and recognize the Bernoulli as the single-trial special case.

> #### 60-second micro-callout
>
> **Bernoulli**$(p)$ **≡** **Binomial**$(n = 1, p)$.
> Each person’s yes/no is a Bernoulli; counts across a small group follow a Binomial.

**Intuition.** If each individual has probability $p$ of “yes” (independent trials, same $p$), then for a group of size $n$, the total number of “yes” responses $X$ is

$$
X \sim \mathrm{Binomial}(n, p),\quad 
P(X=k)=\binom{n}{k}p^{k}(1-p)^{n-k},\; k=0,1,\dots,n.
$$

Special case $n=1$: a single trial is Bernoulli$(p)$, producing 0/1.

---

#### Minimal examples (copy into your notebook)

**A. Single individuals (Bernoulli as Binomial n=1).**


In [ ]:
import numpy as np

p = 0.25                      # probability of "yes" per individual
N = 10_000                    # number of people
single_trials = np.random.binomial(n=1, p=p, size=N)  # 0/1 draws

prop_yes = single_trials.mean()
print(f"Observed P(yes) ≈ {prop_yes:.3f} (theoretical p = {p})")

**B. Small groups (Binomial counts).** Suppose we form groups of size `n=5` and count how many “yes” per group.


In [ ]:
n = 5
G = 5_000                     # number of groups
counts = np.random.binomial(n=n, p=p, size=G)  # total yes per group

# Empirical distribution of counts
vals, freqs = np.unique(counts, return_counts=True)
pmf_emp = freqs / G
print("k   Empirical P(X=k)")
for k, pk in zip(vals, pmf_emp):
    print(f"{k:<2d}  {pk:.3f}")



**C. Common probabilities.** With $X\sim \mathrm{Binomial}(n,p)$:

* “None”: $P(X=0)=(1-p)^n$
* “At least one”: $P(X\ge 1)=1-(1-p)^n$
* “Exactly $k$”: $P(X=k)=\binom{n}{k}p^k(1-p)^{n-k}$


In [ ]:
from math import comb

def binom_pmf(n, p, k):
    return comb(n, k) * (p**k) * ((1-p)**(n-k))

n = 5; p = 0.25
p_none        = (1-p)**n
p_at_least_1  = 1 - (1-p)**n
p_exactly_2   = binom_pmf(n, p, 2)

print(f"P(X=0)         = {p_none:.3f}")
print(f"P(X≥1)         = {p_at_least_1:.3f}")
print(f"P(X=2)         = {p_exactly_2:.3f}")

#### Takeaway

* Treat each person’s response as **Bernoulli(p)**.
* Aggregate to a group of size **n** → total “yes” is **Binomial(n, p)**.
* Use closed-form Binomial probabilities for “none,” “at least one,” and “exactly k,” or simulate to build intuition.